In [1]:
from scenic.simulators.gfootball import rl_interface
from stable_baselines3 import PPO
from scenic.simulators.gfootball.rl_interface import GFScenicEnv
import pretrain_template
from gfootball_impala_cnn import GfootballImpalaCNN
import gym
from tqdm import tqdm
import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy

from torch.utils.data.dataset import Dataset, random_split
import os

/Users/azadsalam/codebase/scenic/src/scenic/core/errors.py:157: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


In [2]:
class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions

    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)


In [3]:
def pretrain_agent(
        student,
        env,
        expert_dataset,
        batch_size=64,
        epochs=10,
        scheduler_gamma=0.7,
        learning_rate=1.0,
        log_interval=100,
        no_cuda=True,
        seed=1,
        test_batch_size=64,
):
    train_size = int(0.8 * len(expert_dataset))

    test_size = len(expert_dataset) - train_size

    train_expert_dataset, test_expert_dataset = random_split(
        expert_dataset, [train_size, test_size]
    )

    print("test_expert_dataset: ", len(test_expert_dataset))
    print("train_expert_dataset: ", len(train_expert_dataset))


    use_cuda = not no_cuda and th.cuda.is_available()
    th.manual_seed(seed)
    device = th.device("cuda" if use_cuda else "cpu")
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    if isinstance(env.action_space, gym.spaces.Box):
        criterion = nn.MSELoss()
    else:
        criterion = nn.CrossEntropyLoss()

    # Extract initial policy
    model = student.policy.to(device)

    def train(model, device, train_loader, optimizer):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            if isinstance(env.action_space, gym.spaces.Box):
                # A2C/PPO policy outputs actions, values, log_prob
                # SAC/TD3 policy outputs actions only
                if isinstance(student, (A2C, PPO)):
                    action, _, _ = model(data)
                else:
                    # SAC/TD3:
                    action = model(data)
                action_prediction = action.double()
            else:
                # Retrieve the logits for A2C/PPO when using discrete actions
                latent_pi, _, _ = model._get_latent(data)
                logits = model.action_net(latent_pi)
                action_prediction = logits
                target = target.long()

            loss = criterion(action_prediction, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    def test(model, device, test_loader):
        model.eval()
        test_loss = 0
        with th.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)

                if isinstance(env.action_space, gym.spaces.Box):
                    # A2C/PPO policy outputs actions, values, log_prob
                    # SAC/TD3 policy outputs actions only
                    if isinstance(student, (A2C, PPO)):
                        action, _, _ = model(data)
                    else:
                        # SAC/TD3:
                        action = model(data)
                    action_prediction = action.double()
                else:
                    # Retrieve the logits for A2C/PPO when using discrete actions
                    latent_pi, _, _ = model._get_latent(data)
                    logits = model.action_net(latent_pi)
                    action_prediction = logits
                    target = target.long()

                test_loss = criterion(action_prediction, target)
        test_loss /= len(test_loader.dataset)
        print(f"Test set: Average loss: {test_loss:.4f}")

    # Here, we use PyTorch `DataLoader` to our load previously created `ExpertDataset` for training
    # and testing
    train_loader = th.utils.data.DataLoader(
        dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
    )
    test_loader = th.utils.data.DataLoader(
        dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
    )

    # Define an Optimizer and a learning rate schedule.
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

    # Now we are finally ready to train the policy model.
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer)
        test(model, device, test_loader)
        scheduler.step()

    # Implant the trained policy network back into the RL student agent
    student.policy = model


In [4]:
cwd = os.getcwd()
rewards = "scoring"
target_scenario_name = f"{cwd}/exp_0_4/academy_run_to_score.scenic"

save_dir = f"{cwd}/saved_models"
logdir = f"{cwd}/tboard/dev"
tracedir = f"{cwd}/game_trace"



In [6]:
#Create Environment

gf_env_settings = {
        "stacked": True,
        "rewards": rewards,
        "representation": 'extracted',
        "players": [f"agent:left_players=1"],
        "real_time": False,
        "action_set": "default"
    }

from scenic.simulators.gfootball.utilities.scenic_helper import buildScenario
scenario = buildScenario(target_scenario_name)
target_env = GFScenicEnv(initial_scenario=scenario, gf_env_settings=gf_env_settings)

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
def mean_reward_random_agent(env, num_trials=1):

    obs = env.reset()
    #env.render()
    num_epi = 0
    total_r = 0
    from tqdm import tqdm
    for i in tqdm(range(0, num_trials)):

        done = False

        while not done:
            action = env.action_space.sample()
            obs, reward, done, info = env.step(action)
            #env.render()
            total_r+=reward
            if done:
                obs = env.reset()
                num_epi +=1

    return total_r/num_epi

In [8]:
#generate expert data

gf_env_settings = {
    "stacked": True,
    "rewards": 'scoring',
    "representation": 'extracted',
    "players": [f"agent:left_players=1"],
    "real_time": False,
    "action_set": "default",#"default" "v2"
}

datagen_scenario_file = f"{cwd}/pretrain/run_to_score_with_behave.scenic"
datagen_scenario = buildScenario(datagen_scenario_file)
from scenic.simulators.gfootball.rl_interface import GFScenicEnv

datagen_env = GFScenicEnv(initial_scenario=datagen_scenario, gf_env_settings=gf_env_settings, use_scenic_behavior_in_step=True)
print("Mean Reward of Scenic Behavior Agent", mean_reward_random_agent(datagen_env, num_trials=5))

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 5/5 [00:02<00:00,  2.33it/s]

Mean Reward of Scenic Behavior Agent 1.0


In [9]:


def generate_expert_data(env, num_interactions=1000):

    expert_observations = []
    expert_actions = []

    obs = env.reset()

    for i in tqdm(range(num_interactions)):
        expert_observations.append(obs)

        obs, reward, done, info = env.step(env.action_space.sample())
        #print(info)
        action = info["action_taken"]
        expert_actions.append(action)

        if done:
            obs = env.reset()
    
    expert_observations = np.array(expert_observations)
    expert_observations = np.moveaxis(expert_observations, [3], [1])
    expert_actions = np.array(expert_actions)
    print("Expert observation shape: ", expert_observations.shape)
    print("Expert actions shape: ", expert_actions.shape)

    np.savez_compressed(
        "expert_data",
        expert_actions=expert_actions,
        expert_observations=expert_observations,
    )
    return expert_observations, expert_actions

expert_observations, expert_actions = generate_expert_data(datagen_env, num_interactions=1000)          

100%|██████████| 1000/1000 [00:06<00:00, 165.51it/s]


Expert observation shape:  (1000, 16, 72, 96)
Expert actions shape:  (1000,)


In [ ]:
#Generate random data for now as proxy to expert data


def gen_dummy_expert_data(num_interactions=1000):
    expert_observations = []
    expert_actions = []

    obs = env.reset()

    for i in tqdm(range(num_interactions)):
        action = env.action_space.sample()
        expert_observations.append(obs)
        expert_actions.append(action)
        obs, reward, done, info = env.step(action)
        if done:
            obs = env.reset()

    expert_observations = np.array(expert_observations)
    print("expert collected obs shape", expert_observations.shape)
    expert_observations = np.moveaxis(expert_observations, [3], [1])
    print("expert obs shape updated: ", expert_observations.shape)
    expert_actions = np.array(expert_actions)

    np.savez_compressed(
        "dummy_expert_data",
        expert_actions=expert_actions,
        expert_observations=expert_observations,
    )

    return expert_observations, expert_actions

dummy_expert_observations, dummy_expert_actions = gen_dummy_expert_data()

In [11]:
#get the PPO object with required parameters
model, parameter_dict = pretrain_template.get_model_and_params(
    env=target_env, ALGO=PPO, features_extractor_class = GfootballImpalaCNN, scenario_name=target_scenario_name,
    logdir=logdir, override_params={}, rewards=rewards)

print("env (from model) observation space: ", model.get_env().observation_space)



Using scoring Parameters
Using cpu device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
device:  cpu
env (from model) observation space:  Box(0, 255, (16, 72, 96), uint8)


In [12]:
loaded_data = np.load("expert_data.npz")

In [13]:
expert_observations = loaded_data["expert_observations"]
expert_actions = loaded_data["expert_actions"]
print(f"Loaded data obs: {expert_observations.shape}, actions: {expert_actions.shape}")
expert_dataset = ExpertDataSet(expert_observations, expert_actions)

Loaded data obs: (1000, 16, 72, 96), actions: (1000,)


In [14]:
pretrain_agent(
    student=model,
    env=target_env,
    expert_dataset=expert_dataset,
    epochs=10
)


test_expert_dataset:  200
train_expert_dataset:  800
Train Epoch: 1 [0/800 (0%)]	Loss: 2.944235
Test set: Average loss: 0.0061
Train Epoch: 2 [0/800 (0%)]	Loss: 1.163416
Test set: Average loss: 0.0040
Train Epoch: 3 [0/800 (0%)]	Loss: 0.894178
Test set: Average loss: 0.0043
Train Epoch: 4 [0/800 (0%)]	Loss: 1.104846
Test set: Average loss: 0.0057
Train Epoch: 5 [0/800 (0%)]	Loss: 1.068910
Test set: Average loss: 0.0057
Train Epoch: 6 [0/800 (0%)]	Loss: 1.111425
Test set: Average loss: 0.0079
Train Epoch: 7 [0/800 (0%)]	Loss: 1.156394
Test set: Average loss: 0.0080
Train Epoch: 8 [0/800 (0%)]	Loss: 1.162453
Test set: Average loss: 0.0058
Train Epoch: 9 [0/800 (0%)]	Loss: 1.271063
Test set: Average loss: 0.0062
Train Epoch: 10 [0/800 (0%)]	Loss: 1.195923
Test set: Average loss: 0.0041


In [15]:
def test_model_performance(env, model, num_trials=1):

    obs = env.reset()
    #env.render()
    num_epi = 0
    total_r = 0
    from tqdm import tqdm
    for i in tqdm(range(0, num_trials)):

        done = False

        while not done:
            action = model.predict(obs, deterministic=True)[0]
            obs, reward, done, info = env.step(action)
            #env.render()
            total_r+=reward
            if done:
                obs = env.reset()
                num_epi +=1

    return total_r/num_epi

print(test_model_performance(target_env, model, num_trials=5))

100%|██████████| 5/5 [00:10<00:00,  2.01s/it]

0.0


In [16]:
#do training
pretrain_template.train(model=model, parameters=parameter_dict,
                            n_eval_episodes=5, total_training_timesteps=5000,
                            eval_freq=5000,
                            save_dir=save_dir, logdir=logdir, dump_info={"rewards": rewards})

Logging to /Users/azadsalam/codebase/scenic/rl_training/tboard/dev/HM_23_59__DM_29_3_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81.6     |
|    ep_rew_mean     | 0.667    |
| time/              |          |
|    fps             | 78       |
|    iterations      | 1        |
|    time_elapsed    | 26       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 80           |
|    ep_rew_mean          | 0.647        |
| time/                   |              |
|    fps                  | 15           |
|    iterations           | 2            |
|    time_elapsed         | 264          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0018253119 |
|    clip_fraction        | 0.0138       |
|    clip_range           | 0.115        |
|    entropy_l